<a href="https://colab.research.google.com/github/zosimanoz/colab-ml/blob/main/BERT_ENC_DEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q datasets==1.0.2
!pip install -q -U transformers==4.28.0


import datasets
import transformers

In [2]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%cd /content/drive/My Drive/bert/

/content/drive/My Drive/bert


In [5]:
import pandas as pd

In [6]:
train_data = pd.read_csv("train.csv")
train_data.head()

,article,highlights
0,AAAAAGCCTGTGAAGCACAGAGAGCAGCCAGCCAGAGCTGATGCTC...,AGCAGAGTGG
1,ACCTGCTAACAATTAAGGCCTCCAGGTCTACCCTGCAGCTGGGCCT...,GAGGGCCTGTGAAG
2,ACTGACCATGTGCATCCTCACTGATACCAGTCTTGCCACAGTGTGC...,GGGGCAGAGTATG
3,ATATTACTCAACCGCCTAACAGAACAAAAGCATTCTTGGCTTGATC...,CTGATAAGCTAGC
4,CAACCATCCTACTTGCTCGTGGGCTAGCTGCGGGCGCGTCGCGAGC...,AACTGAATCTTGCCTT


In [7]:
train_data.article.values

array(['AAAAAGCCTGTGAAGCACAGAGAGCAGCCAGCCAGAGCTGATGCTCAATGGCAGAAACTGCTTAGTCACGCTGAAAGGGAGCCAAGGCAAT',
       'ACCTGCTAACAATTAAGGCCTCCAGGTCTACCCTGCAGCTGGGCCTGAGGAGGTCCTCTTGAAAGGAGTGGGTAACAGCGCACTATT',
       'ACTGACCATGTGCATCCTCACTGATACCAGTCTTGCCACAGTGTGCCTTGGAAACTCTTTCACAGGCAGTTATGGTCCCTACAGATAG',
       'ATATTACTCAACCGCCTAACAGAACAAAAGCATTCTTGGCTTGATCTCTAGAGTCCCTTTGAACAATTGGGACGATGTTCACCGAACT',
       'CAACCATCCTACTTGCTCGTGGGCTAGCTGCGGGCGCGTCGCGAGCTCGTGAAGCTGACATGGCTTTCCGAGGGCACAACACGAG',
       'TCCATGTGTCATCATGGACAGGCTTGTAATGCTTCTCGGGCTTTCACATCCCCACATACTGCAGAGAGTAGTCCAAATT',
       'CTGAGAGGAAAAGGCCCTGAAGAACTGGGAATGCTGAGTCAGCTTTCCAGAAGTTCTGTTGTAGAGCGGTG',
       'ACATCAGGGTGAATGTTGAGGAGTTTAATTATTTTGCTTGGTGACAGTGAGATGTTCACAGTATCAGTTAGTGTGC',
       'TCAGACTAAGTGCCAGTTCTGTGAGTGAATGTTCAATGACTGAGGTGCACATTATTTGCCATATCTACTGAAAAGCAATGCTA',
       'GTTGGGAACCCCAGCTGACTACTTAAGTCTCCCCATTAATTTCTGCTTGATCGGCAGTTTTTAACCTGTGGCCTTAACCTTCAGGCTT',
       'TCCATGTGTCATCATGGACAGGCTTGTAATGCTTCTCGGGCTTTCA

In [8]:
test_data = pd.read_csv("test.csv")
test_data.head()

,article,highlights
0,GTTCCCATGGGAGAGTTCTGTAGGACTATAGGTTGGTCGTATATTG...,TTATCATGCATGCATTAG
1,AGATGCAGTCAGCAGGAAATGTCAAGATTTTATGCAAGTGGCTGTC...,CATGGGGTCAA
2,ACATGTCTTGTGCCATGAGCCCAAAGAGACCATGAGTCAGCACACC...,GAGTGTTAGCCAAGTAGATAAA
3,TATCTCCCACATCCATCCAAGTTTTGTGTTTTTCTCAGCCCATTTC...,TCAGTCATTAACATGT
4,CGCATCTCTCAGGAAAAAAGACTCAGCAAAGCCCACACCAATTTGT...,ACAGCTCCCACCATTGTGAACCGCAGA


In [9]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(test_data)

full_dataset = datasets.DatasetDict({"train":train_dataset,"test":val_dataset})
full_dataset

DatasetDict({'train': Dataset(features: {'article': Value(dtype='string', id=None), 'highlights': Value(dtype='string', id=None)}, num_rows: 15), 'test': Dataset(features: {'article': Value(dtype='string', id=None), 'highlights': Value(dtype='string', id=None)}, num_rows: 15)})

In [10]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [11]:
# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["article_len"] = len(tokenizer(x["article"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > tokenizer.model_max_length)
  x["summary_len"] = len(tokenizer(x["highlights"]).input_ids)
  x["summary_longer_64"] = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x

In [12]:
!pip install -q dill==0.3.5.1

In [13]:
sample_size = 15

data_stats = train_dataset.select(range(sample_size)).map(map_to_length, num_proc=1)

data_stats

  0%|          | 0/15 [00:00<?, ?ex/s]

Dataset(features: {'article': Value(dtype='string', id=None), 'article_len': Value(dtype='int64', id=None), 'article_longer_512': Value(dtype='int64', id=None), 'highlights': Value(dtype='string', id=None), 'summary_len': Value(dtype='int64', id=None), 'summary_longer_128': Value(dtype='int64', id=None), 'summary_longer_64': Value(dtype='int64', id=None)}, num_rows: 15)

In [14]:
def compute_and_print_stats(x):
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size,
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats,
  batched=True,
  batch_size=-1,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

Article Mean: 42.2, %-Articles > 512:0.0, Summary Mean:10.533333333333333, %-Summary > 64:0.0, %-Summary > 128:0.0


In [15]:
encoder_max_length=512
decoder_max_length=50

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [16]:
batch_size=4

gene_train_data = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "highlights"]
)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [17]:
gene_train_data

Dataset(features: {'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'decoder_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'decoder_input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 15)

In [18]:
gene_train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [19]:
val_data = val_dataset.select(range(8))

In [20]:
val_data = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "highlights"]
)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [21]:
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [22]:
from transformers import EncoderDecoderModel
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

In [23]:
bert2bert

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [24]:
# set special tokens
bert2bert.config.decoder_start_token_id = 0 #tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

In [25]:
tokenizer.eos_token_id

In [26]:
bert2bert.config
# tokenizer.decoder_start_token_id

EncoderDecoderConfig {
  "_commit_hash": null,
  "decoder": {
    "_name_or_path": "bert-base-uncased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,

In [27]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [28]:
!pip install rouge_score
# load rouge for validation
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [29]:
# !pip install accelerate==0.24.0
# !pip install transformers[torch]

! pip install accelerate -U
# ! pip install -U transformers

In [30]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    logging_steps=2,  # set to 1000 for full training
    save_steps=16,  # set to 500 for full training
    eval_steps=4,  # set to 8000 for full training
    warmup_steps=1,  # set to 2000 for full training
    max_steps=16, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=gene_train_data,
    eval_dataset=val_data
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
4,11.510700,14.912156,0.000000,0.000000,0.000000
8,12.342900,9.756245,0.000000,0.000000,0.000000
12,4.115200,11.399037,0.000000,0.000000,0.000000
16,2.454700,11.828526,0.000000,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no 

TrainOutput(global_step=16, training_loss=7.861534953117371, metrics={'train_runtime': 128.4957, 'train_samples_per_second': 0.498, 'train_steps_per_second': 0.125, 'total_flos': 36807246028800.0, 'train_loss': 7.861534953117371, 'epoch': 4.0})

# Evaluation

In [31]:
import datasets
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("./checkpoint-16")
model.to("cuda")

test_data = val_dataset

# only use 4 training examples for notebook - DELETE LINE FOR FULL TRAINING
test_data = test_data.select(range(1))


batch_size = 4  # change to 64 for full evaluation

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]
label_str = results["highlights"]

rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

print(rouge_output)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

Score(precision=0.0, recall=0.0, fmeasure=0.0)


In [33]:
print(f'Pred: {pred_str} \n Original: {label_str}')

Pred: ['##ctctctgcgcgcccccccgcgc ccgcgcacgcgcgagcgcggcgcaggcgcgggcgccgcgc bbgcgctcgcgc acgcgctgcgcdcgcgcagcgccagcccgcccgccgcgggcgccccggccc ccgcccgagccccccgcccccc ccccgcacccgc cccccccgcccacccccacgccctcccgctccccctcgcccttgcgcncgcgcpcgcgcttgccctgccc acgcccaggccccagcgctaggcgccolgcgcglgcgcayagc'] 
 Original: ['TTATCATGCATGCATTAG']


In [38]:
data = {
    'article': ['GTTGGGAACCCCAGCTG']
}
test_df = pd.DataFrame(data)


print(test_df.head())

test = Dataset.from_dict(test_df)

res = test.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]

pred_str

             article
0  GTTGGGAACCCCAGCTG


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

['##ctctctgcgcgcccccccgcgc ccgcgcacgcgcgagcgcggcgcaggcgcgggcgccgcgc bbgcgctcgcgc acgcgctgcgcdcgcgcagcgccagcccgcccgccgcgggcgccccggccc ccgcccgagccccccgcccccc ccccgcacccgc cccccccgcccacccccacgccctcccgctccccctcgcccttgcgcncgcgcpcgcgcttgccctgccc acgcccaggccccagcgctaggcgccolgcgcglgcgcayagc']